In [1]:
from PIL import Image
import cv2
import numpy as np
from datasets import load_dataset
import random
from tqdm.notebook import tqdm, trange

ds = load_dataset("Chris1/cityscapes")

In [2]:
def RGB2gray(img: np.array):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


def RGB2Lab(img: np.array):
    return cv2.cvtColor(img, cv2.COLOR_BGR2LAB)


def RGB2YCC(img: np.array):
    return cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)


def RGB2HSL(img: np.array):
    return cv2.cvtColor(img, cv2.COLOR_BGR2HLS)

In [3]:
def get_same_color_pixels(img, color):
    return ((img == color * np.ones(img.shape))).astype(np.uint8) * 255

In [4]:
def is_inside(coord, shape):
    return coord[0] >= 0 and coord[0] < shape[0] and coord[1] >= 0 and coord[1] < shape[1]


def tuple_sum(t1, t2):
    if len(t1) != len(t2):
        raise ValueError
    return tuple(t1[i] + t2[i] for i in range(len(t1)))

In [5]:
def intersection_over_union(pred: np.array, true: np.array):
    """
    Computes IntersectionOverUnion metrics
    """
    res = {}
    for c in tqdm(np.unique(pred), desc="calculating IoU"):
        res |= {c: np.vectorize(lambda x, y: x == c and y == c)(pred, true).sum() / (np.vectorize(lambda x, y: x == c or y == c)(pred, true)).sum()}
    wmean = sum(res[c] * (true == c).sum() for c in res)
    res |= {'mean': sum(res.values()) / len(res)}
    res |= {'weighted_mean': wmean / (1 << 21)}
    return res

In [64]:
def process_image(img: np.array, class_map: np.array, criteria):
    seeds = []
    label_dict = {}
    for l in tqdm(np.unique(class_map), desc="finding centroids"):
        seeds += (curr := find_centroids(get_same_color_pixels(class_map, l)))
        if l:
            label_dict |= {tuple(seed.tolist()): l for seed in curr}
    
    return regional_growth(img, label_dict, criteria)

In [28]:
def find_centroids(cmap: np.array):
    *_, centroids = cv2.connectedComponentsWithStats(cmap)
    
    res = []
    for c in centroids.astype(int):
        c = c[::-1]
        if is_inside(c, cmap.shape) and cmap[*c] != 0:
           res.append(c)
    return res

In [63]:
def regional_growth(img: np.array, labels: dict, grow_criteria: callable):
    directions = tuple((x, y) for x in range(-2, 3) for y in range(-2, 3) if x and y)

    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    for current_point, label in tqdm(labels.items(), desc=f"class growing"):
        current_class = [current_point]
        
        for x in range(-4, 5):
            for y in range(-4, 5):
                if is_inside(p := tuple_sum(current_point, (x, y)), img.shape):
                    current_class.append(p)
                    mask[*p] = label
        while current_class:
            current_point = current_class.pop()
            for direction in random.sample(directions, 6):
                new_point = tuple_sum(current_point, direction)
                if is_inside(new_point, img.shape) and not mask[*new_point] and grow_criteria(img[*new_point], img[*current_point]):
                    current_class.append(new_point)
                    mask[*new_point] = label
    
    return mask

In [10]:
def show_image(img: np.array):
    cv2.imshow('window', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [66]:
results = {}
criteria = lambda x, y: max(x[1], y[1]) - min(x[1], y[1]) <= 3

for n in trange(500):
    img_res = process_image(RGB2HSL(np.array(ds['validation'].select([n])['image'][0])),
                            img_map := RGB2gray(np.array(ds['validation'].select([n])['semantic_segmentation'])[0]), criteria)
    results[n] = intersection_over_union(img_res, img_map)
    print(results[n])

  0%|          | 0/500 [00:00<?, ?it/s]

finding centroids:   0%|          | 0/14 [00:00<?, ?it/s]

class growing:   0%|          | 0/66 [00:00<?, ?it/s]

calculating IoU:   0%|          | 0/14 [00:00<?, ?it/s]

{np.uint8(0): np.float64(0.0), np.uint8(1): np.float64(0.14971708946977716), np.uint8(2): np.float64(0.023092267235337458), np.uint8(4): np.float64(0.18880093814130752), np.uint8(7): np.float64(0.0002641644159324764), np.uint8(8): np.float64(0.06993763130522014), np.uint8(11): np.float64(0.06893922219245276), np.uint8(13): np.float64(0.0375), np.uint8(17): np.float64(0.051680820580968005), np.uint8(20): np.float64(0.12752104176592027), np.uint8(21): np.float64(0.035206932474666164), np.uint8(23): np.float64(0.8849290391684316), np.uint8(24): np.float64(0.03650491130332796), np.uint8(26): np.float64(0.001995557714999653), 'mean': np.float64(0.11972068684059581), 'weighted_mean': np.float64(0.06151639994604427)}


...

calculating IoU:   0%|          | 0/20 [00:00<?, ?it/s]

{np.uint8(0): np.float64(0.0), np.uint8(1): np.float64(0.1207261815990618), np.uint8(2): np.float64(0.00451309546882609), np.uint8(4): np.float64(0.3077007640301215), np.uint8(6): np.float64(0.13676286072772897), np.uint8(7): np.float64(0.00013693566918123603), np.uint8(8): np.float64(0.1281583529774103), np.uint8(11): np.float64(0.0367197881087637), np.uint8(12): np.float64(0.01250633767111712), np.uint8(17): np.float64(0.09383928382468096), np.uint8(19): np.float64(0.12537384854647685), np.uint8(20): np.float64(0.2082913940613991), np.uint8(21): np.float64(0.001477311778152968), np.uint8(22): np.float64(0.03453389226338153), np.uint8(23): np.float64(0.0030702751876279283), np.uint8(24): np.float64(0.31822565091610416), np.uint8(25): np.float64(0.04430379746835443), np.uint8(26): np.float64(0.011212689482731677), np.uint8(28): np.float64(0.048004042445679636), np.uint8(33): np.float64(0.13399922168893502), 'mean': np.float64(0.08847778619578675), 'weighted_mean': np.float64(0.02151769

finding centroids:   0%|          | 0/18 [00:00<?, ?it/s]

class growing:   0%|          | 0/152 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [69]:
import pandas as pd

In [101]:
df = pd.DataFrame(results).transpose()
df.to_csv('results.csv')

In [114]:
df.loc[df['mean'] == df[['mean']].max()[0]]

C:\Users\User\AppData\Local\Temp\ipykernel_137492\2257573036.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc[df['mean'] == df[['mean']].max()[0]]


,0,1,2,4,7,8,11,13,17,20,...,27,32,31,28,6,10,12,18,15,30
149,0.0,0.100983,0.006225,0.079796,0.000126,0.380508,0.042639,NaN,0.358132,0.214268,...,NaN,NaN,NaN,NaN,0.495628,NaN,NaN,NaN,NaN,NaN


In [75]:
pd.read_csv('results.csv')

,Unnamed: 0,0,1,2,4,7,8,11,13,17,...,27,32,31,28,6,10,12,18,15,30
0,0,0.000000,0.149717,0.023092,0.188801,0.000264,0.069938,0.068939,0.037500,0.051681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.000000,0.110056,0.028342,0.005549,0.000112,0.054326,0.100591,0.241253,0.050987,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.000000,0.099415,0.036189,0.451519,0.003989,0.144118,0.002527,NaN,0.007274,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.000000,0.112787,0.012471,0.488938,0.000219,0.123388,0.096170,0.278405,0.087824,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.000000,0.127515,0.011580,0.169452,0.000102,0.090969,0.063301,NaN,0.091628,...,0.05156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,158,0.000000,0.088129,0.006900,0.054479,0.000305,0.003563,0.119270,NaN,0.098547,...,NaN,NaN,NaN,NaN,0.009000,NaN,NaN,NaN,NaN,NaN
159,159,0.000000,0.111968,0.008397,0.018586,0.000275,0.147892,0.094150,0.030337,0.059691,...,NaN,0.161034,NaN,NaN,0.028462,NaN,0.488987,NaN,NaN,NaN
160,160,0.000004,0.137674,0.012352,0.064984,0.000180,0.001338,0.101286,0.240066,0.032377,...,NaN,NaN,NaN,NaN,0.019597,NaN,0.020682,NaN,NaN,NaN
161,161,0.000001,0.087263,0.098156,0.050554,0.000127,0.021749,0.128762,0.004767,0.049093,...,NaN,NaN,NaN,NaN,0.047158,NaN,NaN,NaN,NaN,NaN


In [96]:
img_res = process_image(img := RGB2HSL(imgRGB := np.array(ds['validation'].select([467])['image'][0])),
                        img_map := RGB2gray(np.array(ds['validation'].select([467])['semantic_segmentation'])[0]), criteria)

finding centroids:   0%|          | 0/13 [00:00<?, ?it/s]

class growing:   0%|          | 0/81 [00:00<?, ?it/s]

In [97]:
show_image(imgRGB[:, :, ::-1])
show_image(img_map)

In [98]:
show_image(img_res)
show_image(imgRGB[:, :, ::-1] * np.array([bl := (img_res == 0), bl, bl]).transpose(1,2,0))

In [99]:
results[467] = intersection_over_union(img_res, img_map) 

calculating IoU:   0%|          | 0/13 [00:00<?, ?it/s]

In [100]:
results[467]

{np.uint8(0): np.float64(0.0),
 np.uint8(1): np.float64(0.06611614526106291),
 np.uint8(4): np.float64(0.30271457535480967),
 np.uint8(7): np.float64(0.0005304551025593063),
 np.uint8(8): np.float64(0.04494591570309586),
 np.uint8(11): np.float64(0.10049220156346379),
 np.uint8(13): np.float64(0.0023958244202960553),
 np.uint8(17): np.float64(0.05053902544200086),
 np.uint8(20): np.float64(0.21052631578947367),
 np.uint8(24): np.float64(0.018175880039191742),
 np.uint8(25): np.float64(0.0021279863106860013),
 np.uint8(26): np.float64(0.18384957419549325),
 np.uint8(33): np.float64(0.03564533815194057),
 'mean': np.float64(0.07831224902569799),
 'weighted_mean': np.float64(0.05327988613481331)}

In [31]:
c_img = img_map.copy()

seeds = []
for l in np.unique(img_map):
    seeds += find_centroids(get_same_color_pixels(img_map, l))

for c in seeds:
    print(c)
    if c_img[*c] == 0:
        for x in range(-2, 3):
            for y in range(-2, 3):
                c_img[c[0] + x, c[1] + y] = 255
show_image(c_img)

[275 798]
[315 793]
[ 950 1319]
[114   9]
[  12 1597]
[ 917 2040]
[1000  731]
[ 44 963]
[ 221 1792]
[ 238 1234]
[317 788]
[317 775]
[348 667]
[328 707]
[331 791]
[379 994]
[378 960]
[371 628]
[384 656]
[395 672]
[396 684]
[401 748]
[395 698]
[395 722]
[397 666]
[415 484]
[ 414 1138]
[ 415 1154]
[419 965]
[418 971]
[732 985]
[591 209]
[ 420 1296]
[ 426 1419]
[ 439 1777]
[  32 1046]
[ 173 1244]
[ 187 1409]
[ 178 1722]
[ 149 1040]
[  96 1091]
[ 160 1094]
[ 219 1097]
[ 286 1045]
[ 278 1100]
[ 242 1387]
[ 249 1427]
[262 975]
[ 271 1787]
[ 231 1236]
[ 269 1238]
[269 820]
[293 861]
[341 771]
[ 346 1292]
[367 682]
[336 935]
[370 748]
[381 937]
[273 813]
[272 827]
[381 611]
[326 935]
[355 934]
[349 976]
[232 417]
[ 14 903]
[275 995]
[ 357 1823]
[345 776]
[344 694]
[357 696]
[359 680]
[ 385 1417]
[ 385 1475]
[ 389 1331]
[379 684]
[379 697]
[ 410 1745]
[ 410 1309]
[ 412 1419]
[ 529 1703]
[545 163]
[123 933]
[254 805]
[458 615]
[428 903]
[396 846]
[ 351 1184]
[ 360 1104]
[ 369 1045]


In [ ]:
cv2.imwrite('source.jpg', imgRGB)
cv2.imwrite('mask.jpg', img_res)
cv2.imwrite('compare.jpg', img[:, :, 0] * (img_res == 0))

In [ ]:
cv2.imwrite('centroids.jpg', c_img)